# Day 08. Exercise 03
# Overfitting

## 0. Imports

In [210]:
import pandas as pd
import numpy as np
import joblib

In [211]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [212]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

## 1. Preprocessing

1. Считайте файл `dayofweek.csv` в кадр данных.
2. Используя `train_test_split` с параметрами `test_size=0.2`, `random_state=21` получить `X_train`, `y_train`, `X_test`, `y_test`.
3. Используя, например, `value_counts()`, проверьте, похоже ли распределение классов в train и test.
4. Используйте дополнительный параметр `stratify=` и снова проверьте распределение, теперь оно должно быть более или менее похожим в обоих наборах данных.

In [213]:
df = pd.read_csv('../data/dayofweek.csv')
df.head()

,numTrials,hour,dayofweek,user_4,user_17,user_30,user_2,user_14,user_12,user_8,...,laba04,laba04s,lab03,lab03s,lab02,code_rvw,laba05,lab05s,laba06,laba06s
0,-0.788667,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.756764,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.724861,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.692958,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.661055,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [214]:
x = df.drop('dayofweek', axis=1)
y = df['dayofweek']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=21)

In [215]:
y_train.value_counts()

dayofweek
3    313
6    287
1    222
5    216
2    125
0    105
4     80
Name: count, dtype: int64

In [216]:
y_test.value_counts()

dayofweek
3    83
6    69
5    55
1    52
0    31
4    24
2    24
Name: count, dtype: int64

In [217]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, train_size=0.8, random_state=21, stratify=y_train)

In [218]:
y_train.value_counts()

dayofweek
3    250
6    229
1    178
5    173
2    100
0     84
4     64
Name: count, dtype: int64

In [219]:
y_test.value_counts()

dayofweek
3    63
6    58
1    44
5    43
2    25
0    21
4    16
Name: count, dtype: int64

## 2. Baseline models

1. Обучите точно такие же базовые модели из предыдущего упражнения и рассчитайте точность на тестовом наборе данных со стратификацией.
2. Все ли модели показали одинаковые значения метрики? Какая из них имеет наибольшую разницу при сравнении текущего упражнения с предыдущим? Поместите ответ в ячейку markdown в конце раздела.

### a. Logreg

In [220]:
model = LogisticRegression(random_state=21, fit_intercept=False)
model.fit(x_train, y_train)
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

accuracy = 0.5777777777777777


### b. SVM

In [221]:
model = SVC(kernel='poly', probability=True, random_state=21)
model.fit(x_train,y_train)
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

accuracy = 0.7518518518518519


### c. Decision tree

In [222]:
model = DecisionTreeClassifier(max_depth=20, random_state=21)
model.fit(x_train, y_train)
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

accuracy = 0.8703703703703703


### d. Random forest

In [223]:
model = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=21)
model.fit(x_train, y_train)
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

accuracy = 0.9074074074074074


Во всех случаях точность уменьшилась. наибольшую разницу имеет Decision tree точность которго упала на 11%

## 3. Crossvalidation

Мы могли бы поиграть с параметрами модели, пытаясь добиться большей точности на тестовом наборе данных, но это плохая практика. Это снова приведет нас к чрезмерной подгонке. Тестовый набор данных нужен только для проверки качества конечной модели.

Но есть и другой способ решения проблемы - кроссвалидация. Он не использует тестовый набор данных, а создает еще одно разбиение обучающего набора данных. Опять же, есть разные способы, но общим является то, что есть валидационный набор данных, который используется для оптимизации гиперпараметров.

1. С помощью `cross_val_score` с `cv=10` рассчитайте среднюю точность и стандартное отклонение для каждой модели, которую вы использовали ранее (logreg с `solver='liblinear'`, SVC, дерево решений, случайный лес).

### a. Logreg

In [224]:
model = LogisticRegression(random_state=21, fit_intercept=False, solver='liblinear')
scores = cross_val_score(model, x_train, y_train, cv=10)
print(f'average accuracy = {scores.mean()} | standard deviation = {scores.std()}')

average accuracy = 0.5668310834198684 | standard deviation = 0.0383971325355844


### b. SVM

In [225]:
model = SVC(kernel='poly', probability=True, random_state=21)
scores = cross_val_score(model, x_train, y_train, cv=10)
print(f'average accuracy = {scores.mean()} | standard deviation = {scores.std()}')

average accuracy = 0.7894167532017999 | standard deviation = 0.04919379240385882


### c. Decision tree

In [226]:
model = DecisionTreeClassifier(max_depth=20, random_state=21)
scores = cross_val_score(model, x_train, y_train, cv=10)
print(f'average accuracy = {scores.mean()} | standard deviation = {scores.std()}')

average accuracy = 0.8673502942194531 | standard deviation = 0.03993392385772471


### d. Random forest

In [227]:
model = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=21)
scores = cross_val_score(model, x_train, y_train, cv=10)
print(f'average accuracy = {scores.mean()} | standard deviation = {scores.std()}')

average accuracy = 0.8924368293527172 | standard deviation = 0.03120440587133007


## 4. Optimization

1. Выберите лучшую модель и немного поиграйте с параметрами на кросс-валидации, найдите достаточно хороший параметр или комбинацию параметров.
2. Рассчитайте точность для итоговой модели на тестовом наборе данных.
3. Постройте график, отображающий топ-10 наиболее важных признаков для данной модели.
4. Сохраните модель с помощью `joblib`.
5. Загрузите модель, сделайте прогнозы для тестового набора данных и рассчитайте точность.

In [228]:
def plot_features_bokeh(coef, features, n):
    coef = coef.mean(axis=0)
    coef = coef / coef.sum()

    indices = coef.argsort()[::-1][:n]
    sorted_coef = coef[indices]
    sorted_features = features[indices].tolist()

    source = ColumnDataSource(data=dict(
        features=sorted_features,
        coef=sorted_coef
    ))

    p = figure(
        y_range=sorted_features,
        title=f'Top {n} Features',
        x_axis_label='Feature Importance',
        y_axis_label='Features',
        toolbar_location=None,
        width=800,
        height=400
    )

    p.hbar(
        y='features',
        right='coef',
        height=0.5,
        source=source,
    )

    p.x_range.start = 0

    show(p)

In [232]:
model = RandomForestClassifier(max_depth=25, n_estimators=100, random_state=21)
scores = cross_val_score(model, x_train, y_train, cv=10)
print(f'average accuracy = {scores.mean()} | standard deviation = {scores.std()}')
model.fit(x_train, y_train)
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

average accuracy = 0.8924368293527172 | standard deviation = 0.03120440587133007
accuracy = 0.9074074074074074


In [233]:
coef = pd.DataFrame(model.feature_importances_).T
plot_features_bokeh(coef, x.columns, 10)

In [234]:
joblib.dump(model, 'model.pkl')
model = joblib.load('model.pkl')
predict = model.predict(x_test)
print(f'accuracy = {accuracy_score(y_test, predict)}')

accuracy = 0.9074074074074074
